In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Cargar los datos
data = np.load('c:\CAROLINA\AI_BOOTCAMP\HACKATONE_F5_TELEFÓNICA\processed_data_full4k.npz')
X = data['images']
y = data['labels']

print("Forma de X:", X.shape)
print("Forma de y:", y.shape)
print("Valores únicos en y:", np.unique(y))

# Convertir etiquetas a formato numérico
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)
print("Clases:", le.classes_)

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Preprocesamiento y aumento de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(
    X_train, tf.keras.utils.to_categorical(y_train, num_classes),
    batch_size=32, subset='training')

validation_generator = train_datagen.flow(
    X_train, tf.keras.utils.to_categorical(y_train, num_classes),
    batch_size=32, subset='validation')

# Modelo
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entrenamiento
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=len(X_train) // 160
)

# Evaluación
test_loss, test_acc = model.evaluate(test_datagen.flow(X_test, tf.keras.utils.to_categorical(y_test, num_classes), batch_size=32), steps=len(X_test) // 32)
print(f'Test accuracy: {test_acc}')

# Guardar el modelo y las etiquetas
model.save('modelo_entrenado.h5')
np.save('label_encoder.npy', le.classes_)

<>:12: SyntaxWarning: invalid escape sequence '\C'
<>:12: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9472\2681645675.py:12: SyntaxWarning: invalid escape sequence '\C'
  data = np.load('c:\CAROLINA\AI_BOOTCAMP\HACKATONE_F5_TELEFÓNICA\processed_data_full4k.npz')


Forma de X: (4247, 224, 224, 3)
Forma de y: (4247,)
Valores únicos en y: [0 1 2 3]
Clases: [0 1 2 3]
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 7s 1us/step
Epoch 1/50


c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 85/106 ━━━━━━━━━━━━━━━━━━━━ 15s 746ms/step - accuracy: 0.4736 - loss: 1.1865

c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


106/106 ━━━━━━━━━━━━━━━━━━━━ 144s 768ms/step - accuracy: 0.4802 - loss: 1.1778 - val_accuracy: 0.5149 - val_loss: 1.1169
Epoch 2/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 59s 530ms/step - accuracy: 0.5056 - loss: 1.1250 - val_accuracy: 0.5714 - val_loss: 1.0231
Epoch 3/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 72s 652ms/step - accuracy: 0.5259 - loss: 1.1071 - val_accuracy: 0.5164 - val_loss: 1.1117
Epoch 4/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 57s 510ms/step - accuracy: 0.5170 - loss: 1.1072 - val_accuracy: 0.4286 - val_loss: 1.1019
Epoch 5/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 71s 643ms/step - accuracy: 0.5148 - loss: 1.1225 - val_accuracy: 0.5164 - val_loss: 1.1199
Epoch 6/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 57s 511ms/step - accuracy: 0.5163 - loss: 1.1215 - val_accuracy: 0.4286 - val_loss: 1.5376
Epoch 7/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 69s 632ms/step - accuracy: 0.5131 - loss: 1.1167 - val_accuracy: 0.5134 - val_loss: 1.1261
Epoch 8/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 56s 502ms/step - accuracy: 0.5177 - loss: 1.1147 - va

Test accuracy: 0.536057710647583
